Basic model of the implementation with a single iteration

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import random

In [ ]:
class DSO:
	def __init__(self, totalHours, marginalCost, unitSalePrice, maxPriceReference):
		self.totalHours = totalHours
		self.numberOfAggregators = 0
		self.marginalCost = marginalCost
		self.unitSalePrice = unitSalePrice
		self.maxPriceReference = maxPriceReference
		self.aggregators = []
		self.maxUtility = 0
		self.m = 0

	def addAggregator(self, aggregator):
		self.aggregators.append(aggregator)
		self.numberOfAggregators = self.numberOfAggregators + 1

	def getAggregators(self):
		return self.aggregators

	def Sfunction(self, aggregator, tIndex):
		return aggregator.maximumDemand[tIndex] * aggregator.maxPriceReference * (1 - np.exp(-aggregator.preferenceSatisfaction[tIndex] * (aggregator.bestLoadResponse[tIndex] / aggregator.nominalDemand[tIndex])))
	
	def CostFunction(self, tIndex):
		return self.marginalCost[tIndex]
	
	def getLeftLimit(self):
		leftLimit = [0 for i in range(self.totalHours)]
		for i in range(self.totalHours):
			mx = 0
			for j in range(self.numberOfAggregators):
				agg = self.aggregators[j]
				mx = max(mx, agg.preferenceSatisfaction[i] * agg.maxPriceReference * np.exp(-agg.preferenceSatisfaction[i] * (agg.maximumDemand[i] / agg.nominalDemand[i])))
			leftLimit[i] = mx
		return leftLimit
	
	def getRightLimit(self):
		rightLimit = [0 for i in range(self.totalHours)]
		for i in range(self.totalHours):
			mx = 0
			for j in range(self.numberOfAggregators):
				agg = self.aggregators[j]
				mx = max(mx, agg.preferenceSatisfaction[i] * agg.maxPriceReference * np.exp(-agg.preferenceSatisfaction[i] * (agg.minimumDemand[i] / agg.nominalDemand[i])))
			rightLimit[i] = mx
		return rightLimit
	
	def getUnitSalePriceRange(self):
		minPrice = self.marginalCost
		leftLimit = self.getLeftLimit()
		minPrice = np.minimum(minPrice, leftLimit)

		maxPrice = self.maxPriceReference
		rightLimit = self.getRightLimit()
		maxPrice = np.maximum(maxPrice, rightLimit)

		return minPrice, maxPrice

	def generateUnitSalePrice(self):
		minPrice, maxPrice = self.getUnitSalePriceRange()
		for i in range(self.totalHours):
			self.unitSalePrice[i] = random.uniform(minPrice[i], maxPrice[i])

	def generateM(self):
		m = 0
		mx = 0
		for i in range(self.totalHours):
			sum = 0
			for j in range(self.numberOfAggregators):
				agg = self.aggregators[j]
				sum = sum + agg.bestLoadResponse[i]
			mx = max(mx, sum)
		m = random.uniform(mx, mx*10)
		self.m = m

	def utilityFunction(self, theta, omega):
		mx = -1000000000000
		for j in range(self.totalHours):
			sum = 0
			for i in range(self.numberOfAggregators):
				agg = self.aggregators[i]
				sum = sum + self.unitSalePrice[j] * agg.bestLoadResponse[j] - self.CostFunction(j) * agg.bestLoadResponse[j] + omega * self.Sfunction(agg, j)
			sum = sum - theta*self.m
			mx = max(mx, sum)
		return mx

	def getPAR(self):
		sum = 0
		for i in range(self.totalHours):
			for j in range(self.numberOfAggregators):
				sum = sum + self.aggregators[j].bestLoadResponse[i]
		return (self.m * self.totalHours) / (sum)

class Aggregator:
	def __init__(self, totalHours, nominalDemand, minimumDemand, maximumDemand, maxPriceReference, preferenceSatisfaction):
		self.totalHours = totalHours
		self.nominalDemand = nominalDemand
		self.minimumDemand = minimumDemand
		self.maximumDemand = maximumDemand
		self.maxPriceReference = maxPriceReference
		self.preferenceSatisfaction = preferenceSatisfaction
		self.currentUtility = 0
		self.bestLoadResponse = [0 for i in range(totalHours)]

	def utilityFunction(self, DSO):
		sum = 0
		for i in range(DSO.totalHours):
			sum = sum + DSO.Sfunction(self, i) - (DSO.unitSalePrice[i] * self.bestLoadResponse[i])
		return sum

	def optimalDR(self, DSO, j):
		self.bestLoadResponse[j] = ((self.nominalDemand[j] / self.preferenceSatisfaction[j]) * (np.log((self.preferenceSatisfaction[j] * self.maxPriceReference) / DSO.unitSalePrice[j])))

In [ ]:
datasetPath = '../dataset/'

In [ ]:
numOfLA = 3
numOfHours = 50

In [ ]:
nominalDemands = []
for i in range (0, 3):
	letter = chr(ord('A') + i)
	fileName = 'Home' + letter + '_hourly.csv'
	df = pd.read_csv(datasetPath + fileName)
	df = df[0:numOfHours]
	# Power is in kW
	nominalDemands.append(df['Power'].tolist())

In [ ]:
LAs = []
maxPriceReference = 100
for i in range(numOfLA):
	nominalDemand = nominalDemands[i]
	maxNominalDemand = max(nominalDemand)
	minimumDemand = np.random.uniform(0.0001, nominalDemand, numOfHours)
	maximumDemand = np.random.uniform(nominalDemand, maxNominalDemand+1, numOfHours)
	preferenceSatisfaction = np.random.uniform(0.1, 15, numOfHours)
	LAs.append(Aggregator(numOfHours, nominalDemand, minimumDemand, maximumDemand, maxPriceReference, preferenceSatisfaction))


In [ ]:
marginalCost = np.random.randint(1, 7, numOfHours) # cent/kW
unitSalePrice = np.random.randint(1, 7, numOfHours) # cent/kWh
dso = DSO(numOfHours, marginalCost, unitSalePrice, maxPriceReference)

In [ ]:
# append all the LAs to the DSO
for i in range(numOfLA):
	dso.addAggregator(LAs[i])

In [ ]:
# for each aggregator, find the optimal DR for each hour
for i in range(numOfLA):
	agg = LAs[i]
	for j in range(numOfHours):
		agg.optimalDR(dso, j)

In [ ]:
# generate unit sale price for the DSO
dso.generateUnitSalePrice()

In [ ]:
# generate m for the DSO
dso.generateM()

In [ ]:
# utility function for the DSO
theta = 0.5
omega = 0.5
print("utility function: ", dso.utilityFunction(theta, omega))

In [ ]:
# print Unit Sale Price
print("Unit Sale Price: ", dso.unitSalePrice)

In [ ]:
# print utility for each house
for i in range(numOfLA):
	print("LA", i)
	ut = LAs[i].utilityFunction(dso)
	print("utility: ", ut)

In [ ]:
# print PAR
print("PAR: ", dso.getPAR())

In [ ]:
# plot nominal demand of each LA
for i in range(numOfLA):
	plt.plot(LAs[i].nominalDemand)
plt.legend([chr(ord('A') + i) for i in range(numOfLA)])
plt.show()

In [ ]:
# plot user pref of each LA
plt.figure(figsize=(50, 12), dpi=300)
for i in range(numOfLA):
	plt.plot(LAs[i].preferenceSatisfaction)
plt.legend([chr(ord('A') + i) for i in range(numOfLA)])
plt.show()

In [ ]:
# plot marginal cost
plt.plot(dso.marginalCost)
plt.show()

In [ ]:
# plot bestLoadResponse of each LA
plt.figure(figsize=(50, 12), dpi=300)
for i in range(numOfLA):
	plt.plot(LAs[i].bestLoadResponse)
plt.legend([chr(ord('A') + i) for i in range(numOfLA)])
plt.show()

In [ ]:
# # save the variables for each LA
# for i in range(numOfLA):
# 	letter = chr(ord('A') + i)
# 	fileName = 'Home' + letter + '_hourly.csv'
# 	df = pd.read_csv(datasetPath + fileName)
# 	df = df[0:numOfHours]
# 	df['nominalDemand'] = LAs[i].nominalDemand
# 	df['minimumDemand'] = LAs[i].minimumDemand
# 	df['maximumDemand'] = LAs[i].maximumDemand
# 	df['preferenceSatisfaction'] = LAs[i].preferenceSatisfaction
# 	df['bestLoadResponse'] = LAs[i].bestLoadResponse
# 	df.to_csv(datasetPath + 'Home' + letter + '_hourly_basic_results.csv', index=False)

In [ ]:
# # save the variables for the DSO
# df = pd.DataFrame()
# df['marginalCost'] = dso.marginalCost
# df['unitSalePrice'] = dso.unitSalePrice
# df['PAR'] = dso.getPAR()
# df.to_csv(datasetPath + 'DSO_hourly_basic_results.csv', index=False)